In [221]:
# FLASK_APP=dash_app/app.py flask run
import pandas as pd
import numpy as np
from flask import Flask, jsonify, render_template
import sqlite3
import re

In [222]:
conn = sqlite3.connect("dash_app/db/js_overload.sqlite")
cur = conn.cursor()

In [223]:
genders_query = "SELECT Gender, COUNT(Gender) FROM jso11k GROUP BY Gender"

## Respondent Count by Gender

In [224]:
cur.execute(genders_query)
rows = cur.fetchall()

In [225]:
rows

[(None, 0),
 ('Man', 9588),
 ('Man;Non-binary, genderqueer, or gender non-conforming', 23),
 ('Non-binary, genderqueer, or gender non-conforming', 70),
 ('Woman', 813),
 ('Woman;Man', 15),
 ('Woman;Man;Non-binary, genderqueer, or gender non-conforming', 8),
 ('Woman;Non-binary, genderqueer, or gender non-conforming', 20)]

In [226]:
gender_data = []
for row in rows:
    temp_dict = {}
    temp_dict["gender"] = row[0]
    temp_dict["gender_count"] = row[1]
    gender_data.append(temp_dict)

# return jsonify(gender_data)

In [227]:
gender_data

[{'gender': None, 'gender_count': 0},
 {'gender': 'Man', 'gender_count': 9588},
 {'gender': 'Man;Non-binary, genderqueer, or gender non-conforming',
  'gender_count': 23},
 {'gender': 'Non-binary, genderqueer, or gender non-conforming',
  'gender_count': 70},
 {'gender': 'Woman', 'gender_count': 813},
 {'gender': 'Woman;Man', 'gender_count': 15},
 {'gender': 'Woman;Man;Non-binary, genderqueer, or gender non-conforming',
  'gender_count': 8},
 {'gender': 'Woman;Non-binary, genderqueer, or gender non-conforming',
  'gender_count': 20}]

## Languages frequency in Men

In [269]:
conn = sqlite3.connect("dash_app/db/js_overload.sqlite")
cur = conn.cursor()

queries = {
        "man": '''
            SELECT LanguageWorkedWith, COUNT(LanguageWorkedWith)
            FROM jso11k WHERE Gender = 'Man' and LanguageWorkedWith > 0
            GROUP BY LanguageWorkedWith ORDER BY COUNT(LanguageWorkedWith)
        ''',
        "woman": '''
            SELECT LanguageWorkedWith, COUNT(LanguageWorkedWith)
            FROM jso11k
            WHERE Gender = 'Woman' and LanguageWorkedWith > 0
            GROUP BY LanguageWorkedWith ORDER BY COUNT(LanguageWorkedWith) DESC
        ''',
        "other": '''
            SELECT LanguageWorkedWith, COUNT(LanguageWorkedWith)
            FROM jso11k
            WHERE Gender NOT IN ('Man', 'Woman') and LanguageWorkedWith > 0
            GROUP BY LanguageWorkedWith ORDER BY COUNT(LanguageWorkedWith) DESC
        '''
    }

languages_by_gender = []

for key, value in queries.items():

    cur.execute(value)
    rows = cur.fetchall()

    language_freq = {}

    for row in rows:
        languages = row[0].split(';')   
        for item in languages:
            if item in language_freq:
                language_freq[item] += int(row[1])
            else:
                language_freq[item] = int(row[1])

    languages_by_gender.append({key: language_freq})

In [270]:
languages_by_gender

[{'man': {'Assembly': 648,
   'Bash/Shell/PowerShell': 3528,
   'C#': 3061,
   'Go': 816,
   'HTML/CSS': 6014,
   'JavaScript': 6419,
   'PHP': 2489,
   'Python': 3971,
   'Ruby': 749,
   'SQL': 5326,
   'TypeScript': 2060,
   'VBA': 533,
   'Swift': 586,
   'Java': 3921,
   'R': 564,
   'Scala': 383,
   'Objective-C': 430,
   'Rust': 303,
   'WebAssembly': 113,
   'Other(s):': 865,
   'C++': 2317,
   'Clojure': 139,
   'Dart': 168,
   'Elixir': 139,
   'Erlang': 86,
   'F#': 123,
   'Kotlin': 591,
   'C': 1995}},
 {'woman': {'HTML/CSS': 530,
   'JavaScript': 517,
   'Python': 313,
   'C#': 180,
   'SQL': 386,
   'Java': 284,
   'PHP': 182,
   'TypeScript': 130,
   'Bash/Shell/PowerShell': 225,
   'R': 75,
   'Go': 38,
   'Ruby': 85,
   'C': 121,
   'C++': 144,
   'Kotlin': 30,
   'Assembly': 55,
   'Objective-C': 25,
   'Swift': 44,
   'Other(s):': 58,
   'WebAssembly': 8,
   'Rust': 13,
   'VBA': 38,
   'Clojure': 3,
   'Dart': 6,
   'Erlang': 3,
   'Elixir': 10,
   'Scala': 26,
   '

In [ ]:
def CountFrequency(my_list): 
  
    # Creating an empty dictionary  
    freq = {} 
    for item in my_list: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
  
    for key, value in freq.items(): 
        print ("% d : % d"%(key, value)) 
  
# Driver function 
if __name__ == "__main__":  
    my_list =[1, 1, 1, 5, 5, 3, 1, 3, 3, 1, 4, 4, 4, 2, 2, 2, 2] 

In [ ]:
app = Flask(__name__)

@app.route("/")
def index():
    """Return the homepage."""
    return render_template("index.html")

@app.route("/columns")
def names():
    """Return Column names from  a random sample  n = 11000 rows from the original dataset."""

    conn = sqlite3.connect("dash_app/db/js_overload.sqlite")
    cur = conn.cursor()
    cur.execute("PRAGMA table_info(jso11k)")
    rows = cur.fetchall()
    column_names = [i[1] for i in rows]
    return jsonify(column_names)

    # COUNTRIES API
@app.route("/countries")
def countries():
    """
     Return a list of number of survey respondents by country
    """
    conn = sqlite3.connect("dash_app/db/js_overload.sqlite")
    cur = conn.cursor()

    query_string = '''
        SELECT Country, count(country) FROM jso11k
        GROUP BY Country
        ORDER BY COUNT(Country) DESC
    '''

    cur.execute(query_string)
    rows = cur.fetchall()

    country_data = []

    for row in rows:
        tempDict = {}
        tempDict["country"] = row[0]
        tempDict["respondentCount"] = int(row[1])
        # tempDict[row[0]] = int(row[1])
        country_data.append(tempDict)

    return jsonify(country_data)

####################################################
####################################################

@app.route("/genders")
def genders():
    """
     Return a list of respondents count and percentages out of those who answered the Gender question
    """
    conn = sqlite3.connect("dash_app/db/js_overload.sqlite")
    cur = conn.cursor()

    genders_query = "SELECT Gender, COUNT(Gender) FROM jso11k GROUP BY Gender"

    cur.execute(genders_query)
    rows = cur.fetchall()

    gender_data = []
    for row in rows:
        temp_dict = {}
        temp_dict["gender"] = row[0]
        temp_dict["gender_count"] = row[1]
        gender_data.append(temp_dict)

    return jsonify(gender_data)
####################################################
####################################################

    # MAP CHART ROUTE
    # @TODO route for the map chart
# THE CSS STYLE IS JUST A DEMO
@app.route("/mapChart")
def mapChart():
    """Return the mapChart page."""
    return render_template("mapChart.html")
###################################################
####################################################
    # LINE CHART ROUTE
    # @TODO route for the LINE chart
# THE CSS STYLE IS JUST A DEMO
@app.route("/lineChart")
def lineChart():
    """Return the lineChart page."""
    return render_template("lineChart.html")

####################################################
####################################################
    # LINE CHART ROUTE
    # @TODO route for the donutChart chart
# THE CSS STYLE IS JUST A DEMO
@app.route("/donutChart")
def donutChart():
    """Return the lineChart page."""
    return render_template("donutChart.html")
    
####################################################
####################################################

    # BAR CHART ROUTE
    # @TODO route for the  BAR chart
# THE CSS STYLE IS JUST A DEMO
@app.route("/barChart")
def barChart():
    """Return the barChart page."""
    return render_template("barChart.html")####################################################

if __name__ == "__main__":
    app.run()
